# Religious counselor with Claude LLM



## Overview

*This notebook demonstrates the usecase of a religious counselor. We 1.create emebeddings of publically available versions of cited religious books, 2. use RAG (Retrieval Augmented Generation) to make Claude LLM aware on the context and then demonstrate a hypothetical back and forth conversation using chat API. The model is notably on point in terms of keeping the conversation relevant to the religious and spiritual context. 


## Use Cases

1. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions
2. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.



## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "wailing_mckinsey"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
  Using profile: wailing_mckinsey
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [31]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:


ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: Invalid prompt: prompt must end with "

Assistant:" turn

What happens here? We said "Hi there!" and the model spat out a several conversations. This is due to the fact that the default prompt used by Langchain ConversationChain is not well designed for Claude. An [effective Claude prompt](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design) should contain `\n\nHuman:` at the beginning and also contain `\n\nAssistant:` in the prompt sometime after the `\n\nHuman:` (optionally followed by other text that you want to [put in Claude's mouth](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting#use-human-and-assistant-to-put-words-in-claudes-mouth)). Let's fix this.

To learn more about how to write prompts for Claude, check [Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

## Chatbot using prompt template (Langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [50]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="Hi there!"))

 Hello! Nice to meet you. I'm Claude, an AI assistant created by Anthropic to be helpful, harmless,
and honest. How are you doing today?


#### New Questions

Model has responded with intial message, let's ask few questions

In [4]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

 Here are some tips for starting a new garden:

- Decide what you want to grow - Start by thinking about what types of plants you'd like to grow,
whether it's vegetables, herbs, flowers, or a combination. Choose plants suited to your climate and
space available. Some easy options for beginners include tomatoes, zucchini, beans, lettuce, peas,
radishes, sunflowers, and zinnias.

- Prepare the soil - Dig up any grass or weeds where you want to put your garden. Add compost and
other organic matter to enrich the soil and improve drainage. You can also get a soil test done to
see if you need to adjust pH or nutrient levels. Till the soil to break up clumps.

- Make a plan - Sketch out a simple plan for the garden space. Decide on rows or raised beds. Make
sure taller plants are on the north side so they don't shade shorter plants. Rotate plant families
each year to control pests and diseases.

- Start seeds or buy transplants - You can start seeds indoors 4-6 weeks before your last spring
f

#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [5]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

 Yes, the tips I provided for starting a new garden will work well for growing tomatoes. Here are
some more specific tomato growing tips:

- Choose disease-resistant tomato varieties. Some good options for beginners are Better Boy, Early
Girl, Celebrity, and Sweet 100s.

- Start seeds indoors 6-8 weeks before your last expected spring frost. Use a seed starting mix and
grow the seedlings in full sun.

- Harden off tomato seedlings for 7-10 days before transplanting them outside. This means gradually
exposing them to more sun and wind.

- Transplant the seedlings 18-24 inches apart in rows, with 4 feet between rows. Plant them deep,
burying up to 2/3 of the stem to promote root growth.

- Support the plants with cages or stakes as they grow. This prevents the vines from sprawling on
the ground.

- Water at the base of the plants, avoiding wetting the leaves which can cause diseases. Provide 1-2
inches of water per week.

- Mulch around the plants to retain moisture and prevent weeds. Or

#### Finishing this conversation

In [6]:
print_ww(conversation.predict(input="That's all, thank you!"))

 You're very welcome! I'm glad I could provide some useful tips for starting a new garden,
especially focused on growing tomatoes. It was my pleasure to have this friendly conversation and
share gardening advice. Let me know if you have any other questions as you plan and plant your
garden - I'd be happy to offer more details if needed. Wishing you the best of luck with your new
garden endeavors this season!


Claude is still really talkative. Try changing the prompt to make Claude provide shorter answers.

### Interactive session using ipywidgets

The following utility class allows us to interact with Claude in a more natural way. We write out the question in an input box, and get Claude's answer. We can then continue our conversation.

In [42]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Let's start a chat. You can also test the following questions:
1. tell me a joke
2. tell me another joke
3. what was the first joke about
4. can you make another joke on the same topic of the first joke

In [48]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## Chatbot with persona

AI assistant will play the role of a career coach. Role Play Dialogue requires user message to be set in before starting the chat. ConversationBufferMemory is used to pre-populate the dialog

In [9]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="What are the career options in AI?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
</conversation_history>

Here is the human's next reply:
<human_reply>
What are the career options in AI?
</human_reply>

Assistant:


> Finished chain.
 Here are some of the top career options in the field of artificial intelligence (AI):

- Machine Learning Engineer - These engineers develop and implement various machine learning
algorithms and models to enable systems to automatically learn and improve from data. Skills needed
include


In [10]:
print_ww(conversation.predict(input="What these people really do? Is it fun?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
Human: What are the career options in AI?
AI:  Here are some of the top career options in the field of artificial intelligence (AI):

- Machine Learning Engineer - These engineers develop and implement various machine learning algorithms and models to enable systems to automatically learn and improve from data. Skills needed include
</conversation_history>

Here is the human's next reply:
<human_reply>
What these people really do? Is it fun?
</human_reply>

Assistant:


> Finished chain.
 Here are some d

##### Let's ask a question that is not specialty of this Persona and the model shouldn't answer that question and give a reason for that

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## Chatbot with Context 
In this use case we will ask the Chatbot to answer question from some external corpus it has likely never seen before. To do this we apply a pattern called RAG (Retrieval Augmented Generation): the idea is to index the corpus in chunks, then look up which sections of the corpus might be relevant to provide an answer by using semantic similarity between the chunks and the question. Finally the most relevant chunks are aggregated and passed as context to the ConversationChain, similar to providing a history.

We will take a csv file and use **Titan Embeddings Model** to create vectors for each line of the csv. This vector is then stored in FAISS, an open source library providing an in-memory vector datastore. When the chatbot is asked a question, we query FAISS with the question and retrieve the text which is semantically closest. This will be our answer. 

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [6]:
from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [26]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    #"http://triggs.djvu.org/djvu-editions.com/BIBLES/DRV/Download.pdf",
    "https://www.gita-society.com/bhagavad-gita-in-english-source-file.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [27]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [28]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 2403 documents loaded is 3964 characters.
After the split we have 11683 documents more than the original 2403.
Average length among 11683 documents (after split) is 870 characters.


In [29]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings
    )

    print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

vectorstore_faiss_aws: number of elements in the index=11683::


In [ ]:
import pdftotext
import requests
from io import BytesIO

def download_pdf_file(url, filename):
  """Downloads a PDF file from a web URL and saves it to the specified filename."""

  response = requests.get(url)
  if response.status_code != 200:
    raise Exception("Failed to download PDF file from URL: {}".format(url))

  with open(filename, "wb") as f:
    f.write(response.content)

def convert_pdf_to_text(filename):
  """Converts a PDF file to character text format and returns the text."""

  with open(filename, "rb") as f:
    pdf_text = pdftotext.PDF(f)
    
  return "\n\n".join(pdf_text)

def split_text_into_chunks(text, chunk_size, separator):
  """Splits a text into chunks of the specified size with the specified separator."""

  chunks = []
  current_chunk = ""
  for character in text:
    current_chunk += character
    if len(current_chunk) >= chunk_size or character == separator:
      chunks.append(current_chunk)
      current_chunk = ""

  # If the last chunk is not empty, add it to the list.
  if current_chunk:
    chunks.append(current_chunk)

  return chunks

# Publically available and widely cited version of a religious Book - Bible
url = "http://triggs.djvu.org/djvu-editions.com/BIBLES/DRV/Download.pdf"
filename = "Download"

# Make a request to the URL to download the PDF file.
download_pdf_file(url, filename)

# Create a text PDF file.
pdf_text = convert_pdf_to_text(filename)
#docs = split_text_into_chunks(pdf_text, 2000, ",")
# s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
# !aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

#loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
#documents_aws = loader.load() #

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(pdf_text)

#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [32]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("What were the strengths and weaknesses of Arjuna?", llm=cl_llm))

ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: Invalid prompt: prompt must start with "

Human:" turn, prompt must end with "

Assistant:" turn

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [44]:
v = br_embeddings.embed_query("What were the strengths and weaknesses of Arjuna?r")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

[0.76171875, -0.65234375, 0.06591797, -0.15820312, 1.4765625, -0.25195312, 0.025390625, -0.00044441223, -0.984375, 0.40039062]
Destination of unsuccessful yogi
Arjuna said: The faithful who deviates from the path of medita-
tion and fails to attain yogic perfection due to unsubdued mind —
what is the destination o f such a person, O Krishna? (6.37) Does
he not perish like a dispersing cloud, O Krishna, having lost both
worldly and heavenly  pleasures ; supportless and bewildered on
the path of Self -realization? (6.38) O Krishna, only You are able to
completely dispel this doubt of mine because there is none, other
than You, who can dispel this doubt. (6.39) The Supreme Lord
said: There is no destruction , O Arjuna, for a yogi either here or
hereafter. A spiritualist  is never put to grief, My dear friend. (6.40)
The unsuccessful yogi is reborn in the house of the pious and
prosperous after attaining heaven and living there for many years,
or such a yogi i s born in a family of enlight

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [45]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [46]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

Let's chat! ask the chatbot some questions about SageMaker, like:
1. What is SageMaker?
2. What is canvas?

In [47]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

Thank you , that was a nice chat!!


Your mileage might vary, but after 2 or 3 questions you will start to get some weird answers. In some cases, even in other languages.
This is happening for the same reasons outlined at the beginning of this notebook: the default langchain prompts are not optimal for Claude. 
In the following cell we are going to set two new prompts: one for the question rephrasing, and one to get the answer from that rephrased question.

In [51]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

1. What is SageMaker?
2. what is canvas?

In [52]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens_to_sample` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context